In [1]:
import numpy as np
import pandas as pd

# Necessary to import pypwaves_updated.py from parent directory
import sys
sys.path.append('../')
import pypwaves_updated as pw

In [2]:
# Load data 
pls_file = "../../Data/Run_1_partial/10552_NYU_M2_Pulse_Waves_MTA_Resolved - Scanner 1 - 190511_162124_1 - originalpoints.pls"
pulsewave = pw.openPLS(pls_file)

VLR record length:  216
VLR record length:  72
VLR record length:  130
VLR record length:  248
VLR record length:  1184
Table size:  76
LookupTable size:  84
Inner table size:  1024
num_entries:  256
VLR record length:  1184
Table size:  76
LookupTable size:  84
Inner table size:  1024
num_entries:  256
VLR record length:  196
VLR record length:  300
VLR record length:  404
VLR record length:  300
VLR record length:  404
VLR record length:  404
VLR record length:  300
VLR record length:  404
VLR record length:  404
VLR record length:  404
VLR record length:  404
VLR record length:  300


In [11]:
a = pulsewave.vlrs[300001].record.table_dict[0].full_table
len(a)

1024

In [20]:
a[:42]

b'amplitude conversion table for low channel\x00'

In [ ]:
wave = pulsewave.get_waves(5)
wave.plot()

In [ ]:
wave.smooth(7,3)
wave.plot()

In [ ]:
wave = pulsewave.get_waves(50)
wave.plot()

In [ ]:
# pulse = pulsewave.get_pulse(10000)
# pulse.x_target

In [ ]:
# for key in pulsewave.vlrs[34735].record.key_entry_dict.keys():
#     print(pulsewave.vlrs[34735].record.key_entry_dict[key])

In [ ]:
# for key in pulsewave.vlrs[34735].record.key_entry_dict:
#     tiff_tag = pulsewave.vlrs[34735].record.key_entry_dict[key].tiff_tag_location
#     if tiff_tag == 34736:
#         offset_6 = pulsewave.vlrs[34735].record.key_entry_dict[key].value_offset
#         value_6 = pulsewave.vlrs[34736].record[offset_6]
#         pulsewave.vlrs[34735].record.key_entry_dict[key].value = value_6
#     elif tiff_tag == 34737:
#         offset_7 = pulsewave.vlrs[34735].record.key_entry_dict[key].value_offset
#         len_7 = pulsewave.vlrs[34735].record.key_entry_dict[key].count
#         value_7 = pulsewave.vlrs[34737].record[offset_7:(offset_7+len_7)]
#         pulsewave.vlrs[34735].record.key_entry_dict[key].value = value_7
#     else:
#         pulsewave.vlrs[34735].record.key_entry_dict[key].value = pulsewave.vlrs[34735].record.key_entry_dict[key].value_offset

In [ ]:
# for key in pulsewave.vlrs[34735].record.key_entry_dict:
#     print(pulsewave.vlrs[34735].record.key_entry_dict[key].value)